# Fonction et méthodes

Pour créer une fonction en Julia, il suffit de la déclarer comme suit

In [1]:
function hello end

hello (generic function with 0 methods)

Or comme le résultat de la cellule précédente le montre, cette fonction n'a pas encore de **méthode**. Avant d'en rajouter, il convient de passer en revue les différents types d'arguments qu'une fonction peut accepter :

1. Les arguments *positionnels*, c'est à dire identifiés par leur position ;
1. Les arguments *optionnels*, positionnés après les arguments positionnels et qui prennent une valeur par défaut ;
1. Les arguments *mots-clés*, positionnés après un `;` (point-virgule) et identifiés par un symbole.

Pour ajouter des méthodes, il suffit de réemployer la même syntaxe que précédemment:

In [2]:
# 0-argument
function hello()
    println("Hello, World!")
end

hello (generic function with 1 method)

Cette fonction peut-être invokée de la manière suivante :

In [3]:
hello()

Hello, World!


De premier abord, on a l'impression que la fonction de retourne aucun objet. Par convention, Julia retourne le résultat de la dernière ligne de la fonction, à savoir le résultat de la commande `println("Hello, World!")` ici, soit

In [4]:
typeof(println("Hello, World!"))

Hello, World!


Nothing

Autrement dit, la méthode ajouté à la fonction `hello` et qui correspond à 0 arguments est équivalente à
```julia
function hello()
    return println("Hello, World!")
end
```

<div class="alert alert-block alert-info">
<b>Avertissement</b> : Ce comportement peut-être source de bug pour ceux venant du monde C/Fortran dans lequel le code qui suit un `return` est simplement ignoré par le compilateur. D'autres symboles réserver pour le contrôle sont <tt>continue</tt> et <tt>break</tt>.
</div>

Cela signifie donc que `println` renvoie un object de type `Nothing`. En fait, `Nothing` est un type *singleton*, c'est à dire un type qui n'a qu'une seul instance, ici définie dans le module `Base` de Julia par une ligne qui ressemble à
```julia
const nothing = Nothing()
```

La fonction `hello` elle aussi a un type :

In [5]:
typeof(hello)

typeof(hello) (singleton type of function hello, subtype of Function)

On lit ici que la fonction `hello` est de type `typeof(hello)`, un type *singleton* qui hérite du type `Function`.

Le type `Function` est un type abstrait, c'est à dire qu'il ne peut pas être instancier. De plus, en Julia, un type (abstrait ou concret) ne peut hériter que d'un seul type abstrait (ou aucun).

<div class="alert alert-block alert-info">
    <b>Info</b> : Cette limitation, qui simplifie énormément le mécanisme de <i>dispatch</i>, n'est en fait pas si contraignante. En effet, on peut introduire un niveau d'indirection intermédiaire grâce à un système de <i>traits</i> (<i>Holy traits</i>) implémenté grâce à un autre type <i>singleton</i>, <tt>Type{T}</tt>, dont la seule instance est le type <tt>T</tt>.
</div>

On rajoute des méthodes à la fonction `hello` comme suit (noter la syntaxe sur une ligne, strictement équivalentes aux syntaxes précédentes avec et sans `return`, mais sans les délimiteurs `function` et `end`) :

In [6]:
hello(x) = println("Hello, $(x)!")
hello(x, y) = println("Hello, $(x) and $(y)!")

hello (generic function with 3 methods)

À noter l'opérateur d'interpolation (`$`) qui convertit l'objet référencer en chaine de caractère (`String`) et le remplace dans la chaine. On peut enfin généraliser ce comportement grâce à une fonction variadique (noter la concaténation de chaines de caractères réalisée par la fonction `*`) :

In [7]:
hello(x...) = println("Hello, " * "$(join(Base.front(x), ", ")) and $(Base.last(x))!")

hello (generic function with 4 methods)

In [8]:
hello("Pierre", "Paul", "Jacques")

Hello, Pierre, Paul and Jacques!


<div class="alert alert-block alert-info">
    <b>Complément</b> : Seuls des arguments positionnels ont été utilisés jusqu'ici, les cas des arguments optionnels ou à mots-clés sont laissés à tout un chacun.
</div>

# Dispatch et types

Jusqu'ici, aucune annotation de type n'a été employée. **Cela ne signifie pas que les objets n'ont pas de types**. Par exemple :

In [9]:
x = 1.
typeof(x)

Float64

Si on souhaite utiliser un type particulier, on peut invoquer son constructeur explicitement, par exemple pour associer au symbole `x` un nombre flottant en demi-précision (2 octets)

In [10]:
typeof(π), Float16(π)

(Irrational{:π}, Float16(3.14))

Ici, le nombre irrationnel π a été converti en `Float16` grâce à un mécanisme de [promotion et conversion](https://docs.julialang.org/en/v1/manual/conversion-and-promotion/). Cependant, il est préférable de typer le moins strictement possible.

En effet, si une méthode est initialement prévue pour des `Float16`, il y a de grandes chances que son implémentation soit identique pour des `Float32`, `Float64` voire même `Int32`...

Or ce sont tous des sous-types de `Number`, on aurait alors pu définir une fonction comme le cube comme suit :

In [15]:
cube(x::Number) = x ^ 3

cube (generic function with 1 method)

Cette fonction marche même pour des nombres rationnels et des nombres complexes...

In [25]:
cube(2//3), cube(im)

(8//27, 0 - 1im)

mais plus pour des chaines de caractères

In [18]:
square("Ho ! ")

LoadError: MethodError: no method matching square(::String)
[0mClosest candidates are:
[0m  square([91m::Number[39m) at In[11]:1

alors même que l'opération `^` est définie pour les chaines de caractères... Dommage !

In [20]:
"Ho ! " ^ 3

"Ho ! Ho ! Ho ! "

Que se passe t'il lorsque plusieurs méthodes sont applicables ? Par exemple si l'intentation de développeur lorsqu'il applique la fonction `double`

In [26]:
double(x) = 2x

double (generic function with 1 method)

à une chaine de caractères est de la répéter deux fois. Alors il suffit de définir la méthode associée à ce cas particulier :

In [27]:
double(x::String) = x * x

double (generic function with 2 methods)

In [28]:
double(√(1+im)), double("cou")

(2.19736822693562 + 0.9101797211244547im, "coucou")

La méthode à exécuter est déterminée grâce à un ensemble de règles formelles présentées dans l'article :

<div class="csl-entry">Nardelli, F. Z., Belyakova, J., Pelenitsyn, A., Chung, B., Bezanson, J., &#38; Vitek, J. (2018). Julia subtyping: a rational reconstruction. <i>Proceedings of the ACM on Programming Languages</i>, <i>2</i>(OOPSLA), 27. https://doi.org/10.1145/3276483</div>

Certains cas sont indéterminés, par exemple :

In [29]:
foo(x::Int, y) = 2x + y
foo(x, y::Int) = x + 2y

foo (generic function with 2 methods)

Que se passe t'il si les deux arguments sont de type `Int` ?

In [30]:
foo(1, 1)

LoadError: MethodError: foo(::Int64, ::Int64) is ambiguous. Candidates:
  foo(x::Int64, y) in Main at In[29]:1
  foo(x, y::Int64) in Main at In[29]:2
Possible fix, define
  foo(::Int64, ::Int64)

Heureusement, les messages d'erreur sont très compréhensible, il suffit de définir la méthode
```
foo(x::Int, y::Int)
```
pour lever l'indétermination...

# Types de données

Il existe plusieurs types de données, dont la liste est présentée dans la [page dédiée](https://docs.julialang.org/en/v1/manual/types/) de la [documentation officielle](https://docs.julialang.org/) (très bien écrite, soit dit en passant).

On y reviendra dans quelques instants.